In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\TextSummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Projects\\TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

In [12]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, 
                filename=self.config.local_data_file)
            logger.info(f"Downloaded file: {filename} with following headers: {headers}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file} of size {get_size(Path(self.config.local_data_file))}")

    def extract_data(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_data()
except Exception as e:
    raise e

[2024-11-19 16:43:30,040] 16 TextSummarizerLogger - INFO - yaml file config\config.yaml read successfully.
[2024-11-19 16:43:30,062] 16 TextSummarizerLogger - INFO - yaml file params.yaml read successfully.
[2024-11-19 16:43:30,066] 32 TextSummarizerLogger - INFO - Directory artifacts created.
[2024-11-19 16:43:30,070] 32 TextSummarizerLogger - INFO - Directory artifacts/data_ingestion created.
[2024-11-19 16:43:32,776] 10 TextSummarizerLogger - INFO - Downloaded file: artifacts\data_ingestion\data.zip with following headers: Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A45A:1424C:1CA44E:1D838E:673CB221
Accept-Ranges: bytes
Date: Tue,